In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count
from pyspark.sql.window import Window

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

login_details = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/online") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "login_details") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [11]:
from pyspark.sql.functions import lag,sum,when,min,max

login_details\
    .withColumn("status_lag",lag("status",1,"on").over(Window.orderBy(col("times"))))\
    .withColumn("sum_status",sum(when(col("status_lag")=='off',1).otherwise(0)).over(Window.orderBy("times")))\
    .groupby("sum_status")\
    .agg(min("times").alias("log on"),
         max("times").alias("log off"),
         (count("sum_status")-1).alias("total"))\
    .filter(col("total") >= 1)\
    .show()

24/04/07 20:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/07 20:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/07 20:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/07 20:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/07 20:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/07 20:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/07 2

+----------+-------------------+-------------------+-----+
|sum_status|             log on|            log off|total|
+----------+-------------------+-------------------+-----+
|         0|1970-01-01 10:00:00|1970-01-01 10:03:00|    3|
|         1|1970-01-01 10:04:00|1970-01-01 10:06:00|    2|
|         4|1970-01-01 10:09:00|1970-01-01 10:13:00|    4|
|         6|1970-01-01 10:15:00|1970-01-01 10:16:00|    1|
+----------+-------------------+-------------------+-----+

